In [61]:
import pandas as pd
import numpy as np
import datetime
import holidays
pd.options.display.float_format = '{:.2f}'.format

In [62]:
df = pd.read_csv("data/bikes.csv", index_col=0)

<p> Eliminamos la columna index al tratarse de un índice </p>

In [63]:
df.drop(['instant'], axis= 1, inplace= True)

<p> Cambiamos la columna dteday a datetime </p>

In [64]:
df["dteday"] = pd.to_datetime(df["dteday"], format='%d-%m-%Y')

In [65]:
df.head(15)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2018-01-01,spring,0,1,0,6,0,2,14.11,18.18,80.58,10.75,331,654,985
1,2018-01-02,spring,0,1,0,0,0,2,14.90,17.69,69.61,16.65,131,670,801
2,2018-01-03,spring,0,1,0,1,1,1,8.05,9.47,43.73,16.64,120,1229,1349
3,2018-01-04,spring,0,1,0,2,1,1,8.20,10.61,59.04,10.74,108,1454,1562
4,2018-01-05,spring,0,1,0,3,1,1,9.31,11.46,43.70,12.52,82,1518,1600
5,2018-01-06,spring,0,1,0,4,1,1,8.38,11.66,51.83,6.00,88,1518,1606
6,2018-01-07,spring,0,1,0,5,1,2,8.06,10.44,49.87,11.30,148,1362,1510
7,2018-01-08,spring,0,1,0,6,0,2,6.76,8.11,53.58,17.88,68,891,959
8,2018-01-09,spring,0,1,0,0,0,1,5.67,5.81,43.42,24.25,54,768,822
9,2018-01-10,spring,0,1,0,1,1,1,6.18,7.54,48.29,14.96,41,1280,1321


Recategorizamos la columna "yr"

In [66]:
df['yr'] = df['dteday'].apply(lambda x: x.year)

Recategorizamos la columna "mnth"

In [67]:
df['mnth'] = df['dteday'].apply(lambda x: x.strftime("%b"))

Recategorizamos la columna "weekday"

In [68]:
df['weekday'] = df['dteday'].apply(lambda x: x.strftime("%a"))

Recategorizamos la columna "season"

In [69]:
def estacion(fecha):
    
    #Establecemos los periodos de cada estación:
    primavera_inicio = pd.Timestamp(fecha.year, 3, 20)
    primavera_fin = pd.Timestamp(fecha.year, 6, 20)
    verano_inicio = pd.Timestamp(fecha.year, 6, 21)
    verano_fin = pd.Timestamp(fecha.year, 9, 22)
    otono_inicio = pd.Timestamp(fecha.year, 9, 23)
    otono_fin = pd.Timestamp(fecha.year, 12, 20)
    
    #hacemos la conversión de fecha a estación
    if primavera_inicio <= fecha <= primavera_fin:
        return "Primavera"
    elif verano_inicio <= fecha <= verano_fin:
        return "Verano"
    elif otono_inicio <= fecha <= otono_fin:
        return "Otoño"
    else:
        return "Invierno"

# Hacemos el apply:
df["season"] = df["dteday"].apply(estacion)

Recategorizamos "holiday"

In [70]:
lista_festivos = []

fiestas_dc=holidays.US(subdiv='DC', years=[2018, 2019])
for fecha, name in sorted(fiestas_dc.items()):
    lista_festivos.append(str(fecha))

In [71]:
def holidays(fecha):

    if str(fecha).split(" ")[0] in lista_festivos:
        return "festivo"
    
    else:
        return "no festivo"

In [72]:
df["holiday"] = df["dteday"].apply(holidays)

In [73]:
df.groupby("yr")["holiday"].value_counts()

yr    holiday   
2018  no festivo    353
      festivo        12
2019  no festivo    354
      festivo        11
Name: count, dtype: int64

In [74]:
df.sample(10)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
253,2018-09-11,Verano,2018,Sep,no festivo,Tue,0,1,26.79,30.46,71.38,7.71,1633,3413,5046
381,2019-01-17,Invierno,2019,Jan,no festivo,Thu,1,2,15.29,18.21,71.61,23.39,127,2808,2935
246,2018-09-04,Verano,2018,Sep,no festivo,Tue,0,1,29.08,33.27,74.21,13.83,2521,2419,4940
459,2019-04-05,Primavera,2019,Apr,no festivo,Fri,1,1,17.84,21.56,37.42,14.71,1192,5265,6457
473,2019-04-19,Primavera,2019,Apr,no festivo,Fri,1,1,20.43,24.65,61.25,4.42,846,5719,6565
191,2018-07-11,Verano,2018,Jul,no festivo,Wed,1,1,31.26,36.49,63.58,18.92,723,3363,4086
176,2018-06-26,Verano,2018,Jun,no festivo,Tue,0,1,27.88,31.88,51.33,6.33,1920,3385,5305
405,2019-02-10,Invierno,2019,Feb,no festivo,Sun,1,2,11.51,14.68,54.00,7.83,227,3604,3831
566,2019-07-21,Verano,2019,Jul,no festivo,Sun,0,3,24.43,27.50,86.54,14.29,1264,3195,4459
212,2018-08-01,Verano,2018,Aug,no festivo,Wed,1,1,31.64,35.16,55.08,10.50,729,3537,4266


<p> Guardar los cambios en este archivo al final de la sesión </p>

In [75]:
df.to_pickle("data/bikes_limpieza.pkl")